In [1]:
import struct
import re
from dataclasses import dataclass
import numpy as np
from queue import LifoQueue
from threading import Thread
from PIL import Image
import sys

In [2]:
class Chunk:
    def __init__(self, data: bytes | str, chunk_start=0):
        if isinstance(data, str):
            raw_data = open(data, 'rb').read()
        else:
            raw_data = data

        data_start = chunk_start + 8
        header, self.length = struct.unpack('<4sI', raw_data[chunk_start:data_start])
        self.header = header.decode()
        chunk_end = data_start + self.length
        chunk_data = raw_data[data_start:chunk_end]

        pattern_once = re.compile('HEDR|FINF|SINF|CAMR|SEGM|CLTH|ANM2|LGTP|LGTI|LGTS|MATD|MODL')
        pattern_many = re.compile('MATL|MSH2|GEOM')
        if pattern_once.match(self.header):
            self.data = {}
            offset = data_start
            while offset < chunk_end:
                subchunk = Chunk(raw_data, offset)
                offset += subchunk.length + 8
                self.data[subchunk.header] = subchunk
        elif pattern_many.match(self.header):
            self.data = []
            offset = data_start
            if self.header == 'MATL':
                offset += 4
            while offset < chunk_end:
                subchunk = Chunk(raw_data, offset)
                offset += subchunk.length + 8
                self.data.append(subchunk)
        else:
            self.data = chunk_data

    def __getitem__(self, key):
        return self.data[key]

    def __repr__(self, detailed=True):
        data_repr = '<bytes>'
        if isinstance(self.data, dict):
            data_repr = [self[x] for x in self.data]
        return f'Chunk(header={self.header}, length={self.length}, data={data_repr})'
    
    def tree(self, indent=0):
        space = ' ' * indent
        out = space + self.header + ':'

        pattern_txt = re.compile('FINF|TX[0-9]D|PRFX|PRNT|CTEX|NAME')
        pattern_int = re.compile('SHVO|MTYP|MNDX|FLGS|MATI')
        if pattern_txt.match(self.header):
            out += ' ' + self.data.decode().rstrip('\0')
        elif pattern_int.match(self.header):
            out += ' ' + str(struct.unpack('<I', self.data)[0])
        elif isinstance(self.data, dict):
            for key in self.data:
                out += '\n' + self[key].tree(indent + 4)
        elif isinstance(self.data, list):
            for subchunk in self.data:
                out += '\n' + subchunk.tree(indent + 4)
        elif self.length <= 12:
            out += ' ' + ' '.join(f'{x:02x}' for x in self.data)
        else:
            out += f' <len: {self.length}>'
        return out
    
    def filter(self, key):
        if isinstance(self.data, dict):
            return [self[x] for x in self.data if x == key]
        elif isinstance(self.data, list):
            return [x for x in self.data if x.header == key]
        return []
    
    def filter_all(self, key):
        results = self.filter(key)
        if isinstance(self.data, dict):
            for x in self.data:
                results.extend(self[x].filter_all(key))
        elif isinstance(self.data, list):
            for x in self.data:
                results.extend(x.filter_all(key))
        return results
    
    def data_len(self):
        return struct.unpack('<I', self.data[:4])[0]


In [3]:
@dataclass
class Vert:
    pos: np.array
    norm: np.array
    uv: np.array

@dataclass
class Tri:
    v0: Vert
    v1: Vert
    v2: Vert

    def norm(self):
        n = np.cross(self.v1.pos - self.v0.pos, self.v2.pos - self.v0.pos)
        return n / np.sqrt(n.dot(n))
    
    def split(self):
        v01 = Vert(
            (self.v0.pos + self.v1.pos) / 2,
            (self.v0.norm + self.v1.norm) / 2,
            (self.v0.uv + self.v1.uv) / 2
        )
        v12 = Vert(
            (self.v1.pos + self.v2.pos) / 2,
            (self.v1.norm + self.v2.norm) / 2,
            (self.v1.uv + self.v2.uv) / 2
        )
        v20 = Vert(
            (self.v2.pos + self.v0.pos) / 2,
            (self.v2.norm + self.v0.norm) / 2,
            (self.v2.uv + self.v0.uv) / 2
        )
        return [
            Tri(self.v0, v01, v20),
            Tri(v01, self.v1, v12),
            Tri(v20, v12, self.v2),
            Tri(v01, v12, v20)
        ]

    def area(self):
        n = np.cross(self.v1.pos - self.v0.pos, self.v2.pos - self.v0.pos)
        return 0.5 * np.sqrt(n.dot(n))


In [4]:
# file = Chunk('_lvl_pc/side/imp/msh/imp_fly_tiefighter.msh')
file = Chunk('_lvl_pc/side/rep/msh/rep_hover_fightertank.msh')
print(file.tree())

HEDR:
    MSH2:
        SINF:
            NAME: rep_hover_fightertank
            FRAM: 00 00 00 00 01 00 00 00 9f c2 ef 41
            BBOX: <len: 44>
        MATL:
            MATD:
                NAME: default_material
                DATA: <len: 52>
                ATRB: 04 00 00 00
            MATD:
                NAME: material1
                DATA: <len: 52>
                ATRB: 00 1b 00 00
                TX0D: rep_hover_fightertank.tga
                TX1D: rep_hover_fightertank_bump.tga
        MODL:
            MTYP: 0
            MNDX: 0
            NAME: dummyroot
            FLGS: 1
            TRAN: <len: 40>
        MODL:
            MTYP: 3
            MNDX: 1
            NAME: bone_root
            PRNT: dummyroot
            FLGS: 1
            TRAN: <len: 40>
        MODL:
            MTYP: 3
            MNDX: 2
            NAME: bone_tankski_R
            PRNT: bone_root
            FLGS: 1
            TRAN: <len: 40>
        MODL:
            MTYP: 3
         

In [5]:
segm = max(file.filter_all('SEGM'), key=lambda x: x.length)
print(segm.tree())

SEGM:
    MATI: 1
    POSL: <len: 36964>
    WGHT: <len: 98564>
    NRML: <len: 36964>
    UV0L: <len: 24644>
    NDXL: <len: 33748>
    NDXT: <len: 12660>
    STRP: <len: 7120>


In [6]:
vert_count = segm['POSL'].data_len()
assert vert_count == segm['NRML'].data_len() and vert_count == segm['UV0L'].data_len()
posl = struct.iter_unpack('<3f', segm['POSL'].data[4:4+12*vert_count])
nrml = struct.iter_unpack('<3f', segm['NRML'].data[4:4+12*vert_count])
uv0l = struct.iter_unpack('<2f', segm['UV0L'].data[4:4+8*vert_count])
verts = [Vert(np.array(pos), np.array(norm), np.array(uv)) for pos, norm, uv in zip(posl, nrml, uv0l)]
assert(len(verts) == vert_count)
vert_count

3080

In [7]:
tri_count = segm['NDXT'].data_len()
tris = list(struct.iter_unpack('<3H', segm['NDXT'].data[4:4+tri_count*6]))
filename = './out/' + file.filter_all('SINF')[0]['NAME'].data.decode().rstrip('\0') + '.obj'
with open(filename, 'w') as out_file:
    for v in verts:
        out_file.write(f'v {v.pos[0]} {v.pos[1]} {v.pos[2]}\n')
        out_file.write(f'vt {v.uv[0]} {v.uv[1]}\n')
        out_file.write(f'vn {v.norm[0]} {v.norm[1]} {v.norm[2]}\n')
    for v0, v1, v2 in tris:
        v0, v1, v2 = v0 + 1, v1 + 1, v2 + 1
        out_file.write(f'f {v0}/{v0}/{v0} {v1}/{v1}/{v1} {v2}/{v2}/{v2}\n')
filename

'./out/rep_hover_fightertank.obj'

In [8]:
tri_count = segm['NDXT'].data_len()
tris = (Tri(verts[x[0]], verts[x[1]], verts[x[2]]) for x in struct.iter_unpack('<3H', segm['NDXT'].data[4:4+tri_count*6]))

area_target = 0.0005

q = LifoQueue()
for tri in tris:
    q.put(tri)

tris = []
counter = 0
while q.qsize() > 0:
    tri = q.get()
    if tri.area() > area_target:
        new_tris = tri.split()
        for t in new_tris:
            q.put(t)
    else:
        tris.append(tri)
    counter += 1
    if counter % 10000 == 0:
        print(f'\r{len(tris)} / {q.qsize()}\t\t\t\t', end='', flush=True)
print(f'\r{len(tris)} / {q.qsize()}\t\t\t\t')

424470 / 0							


In [9]:
filename = './out/' + file.filter_all('SINF')[0]['NAME'].data.decode().rstrip('\0') + '.stl'
with open(filename, 'wb') as out_file:
    out_file.write(struct.pack('<80sI', b'\0' * 80, len(tris)))
    size = 84
    for i, tri in enumerate(tris):
        out_file.write(struct.pack('<12fH', *tri.norm(), *tri.v0.pos, *tri.v1.pos, *tri.v2.pos, 0))
        size += 52
        if i % 10000 == 0:
            print(f'\rWriting to {filename} ... {100 * i / len(tris):2.2f}% ({size / (2 ** 20):2.2f} MiB)', end='', flush=True)
print(f'\rWriting to {filename} ... {100:2.2f}% ({size / (2 ** 20):2.2f} MiB)')

Writing to ./out/rep_hover_fightertank.stl ... 100.00% (21.05 MiB)
